In [1]:
from shallowmind.src.model import ModelInterface
from shallowmind.src.data import DataInterface
from shallowmind.api.infer import prepare_inference
import pandas as pd
import torch
from tqdm import tqdm
import pickle
import pytorch_lightning as pl
pl.seed_everything(42)
# hand, d=2
# ckpt = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand/ckpts/exp_name=CL-S2-Tf-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand-cfg=fNIRS_Emo_tramsformer-CL_V0_step2-bs=256-seed=42-val_loss=0.2972.ckpt'
# config = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand/fNIRS_Emo_tramsformer-CL_V0_step2.py'

# hand, d=4
# ckpt = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=4-nh=11bs=128-ls=0.00015-CL_way=pair-label=hand/ckpts/exp_name=CL-S2-Tf-d=4-nh=11bs=128-ls=0.00015-CL_way=pair-label=hand-cfg=fNIRS_Emo_tramsformer-CL_V0_step2-bs=128-seed=42-val_loss=0.2718.ckpt'
# config = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=4-nh=11bs=128-ls=0.00015-CL_way=pair-label=hand/fNIRS_Emo_tramsformer-CL_V0_step2.py'

# relationship, d=2
# ckpt = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=2-nh=11bs=128-ls=0.00015-CL_way=pair-label=relationship/ckpts/exp_name=CL-S2-Tf-d=2-nh=11bs=128-ls=0.00015-CL_way=pair-label=relationship-cfg=fNIRS_Emo_tramsformer-CL_V0_step2-bs=128-seed=42-val_loss=0.0696.ckpt'
# config = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-d=2-nh=11bs=128-ls=0.00015-CL_way=pair-label=relationship/fNIRS_Emo_tramsformer-CL_V0_step2.py'

# # relationship, 0.1, d=2
# ckpt ='/home/xiaowjia/data/Friends_fNIRS/work_dir/CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=relationship/ckpts/exp_name=CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=relationship-cfg=fNIRS_Emo_tramsformer-CL_V0_step2-bs=256-seed=42-val_loss=0.0082.ckpt'
# config = '/home/xiaowjia/data/Friends_fNIRS/work_dir/CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=relationship/fNIRS_Emo_tramsformer-CL_V0_step2.py'

# hand, 0.1, d=2
# ckpt = '/data/xiaowjia/Friends_fNIRS/work_dir/CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand/ckpts/exp_name=CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand-cfg=fNIRS_Emo_tramsformer-CL_V0_step2-bs=256-seed=42-val_loss=0.0733.ckpt'
# config = '/home/xiaowjia/data/Friends_fNIRS/work_dir/CL-S2-Tf-ts=0.1-d=2-nh=11bs=256-ls=0.00015-CL_way=pair-label=hand/fNIRS_Emo_tramsformer-CL_V0_step2.py'

# AllData relationship, d=2
ckpt = '/data/xiaowjia/Friends_fNIRS/work_dir/S2_AllData-Tf-ts=0.3-d=2-nh=11bs=128-ls=0.00015-label=relationship/ckpts/exp_name=S2_AllData-Tf-ts=0.3-d=2-nh=11bs=128-ls=0.00015-label=relationship-cfg=fNIRS_Emo_tramsformer-AllData_V0_step2-bs=128-seed=42-val_f1_score=0.8723.ckpt'
config = '/home/xiaowjia/data/Friends_fNIRS/work_dir/S2_AllData-Tf-ts=0.3-d=2-nh=11bs=128-ls=0.00015-label=relationship/fNIRS_Emo_tramsformer-AllData_V0_step2.py'

data_module, model = prepare_inference(config, ckpt)
data_module.setup()
test_loader = data_module.test_dataloader()
data_table = test_loader.dataset.data_table
model = model.eval()

comet_ml is installed but `COMET_API_KEY` is not set.
Global seed set to 42


The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform


In [2]:
import shap

class Net(torch.nn.Module):
    # This is a wrapper around the model that we want to explain

    def __init__(self, model_interface: ModelInterface):
        super(Net, self).__init__()
        self.model = model_interface.model
        self.model.eval()

    def forward(self, imgs):
        # Avoid in-place operations by using .clone() and ensure that operations
        # are not modifying tensors that are part of the computational graph.

        # Clone imgs to avoid modifying the original tensor
        imgs_cloned = imgs.clone()

        # Concatenating the images
        imgs_concat = torch.cat([imgs_cloned[:, 0, :, :], imgs_cloned[:, 1, :, :]], dim=0)

        # Forward pass through the encoder
        latent = self.model.forward_encoder(imgs_concat)

        # Clone latent to avoid in-place operations
        latent_cloned = latent.clone()

        # Concatenating latent representations
        half_size = latent_cloned.shape[0] // 2
        latent_concat = torch.cat([latent_cloned[:half_size, :], latent_cloned[half_size:, :]], dim=1)

        # Passing through the classifier
        pred = self.model.cls(latent_concat).squeeze(1)

        return pred

In [3]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# SHAP analysis
# device = torch.device('cuda:0')
# model = model.to(device)
data_table = test_loader.dataset.data_table
data_info = test_loader.dataset.data_index_table.reset_index(drop=True)
data_info['idx'] = data_info.index
data = {'seq': [],}
label = []
for batch_idx, d in tqdm(enumerate(test_loader), total=len(test_loader)):
    torch.cuda.empty_cache()
    data['seq'].append(d[0]['seq'])
    label.append(d[1])
data['seq'] = torch.cat(data['seq'], dim=0)
label = torch.cat(label, dim=0)
imgs = data['seq']

100%|██████████| 134/134 [00:00<00:00, 208.76it/s]


In [6]:

data_info_test, data_info_bg = train_test_split(data_info, test_size=0.005, random_state=42, stratify=data_info[['relationship_label', 'hand', 'emotion_label']])
data_info_bg = data_info_bg.reset_index(drop=True)
data_info_test = data_info_test.reset_index(drop=True)
print(data_info_bg.shape, data_info_test.shape)

imgs_bg = data['seq'][data_info_bg['idx'].values]
imgs_test = data['seq'][data_info_test['idx'].values][:3]
label_bg = label[data_info_bg['idx'].values]
label_test = label[data_info_test['idx'].values][:3]

(86, 7) (16942, 7)


In [7]:
device = torch.device('cuda:0')
e = shap.DeepExplainer(Net(model).to(device), imgs_bg.to(device), torch.device('cuda:0'))
shap_values = e.shap_values(imgs_test.to(device))

AssertionError: The SHAP explanations do not sum up to the model's output! This is either because of a rounding error or because an operator in your computation graph was not fully supported. If the sum difference of %f is significant compared to the scale of your model outputs, please post as a github issue, with a reproducible example so we can debug it. Used framework: pytorch - Max. diff: 11.089337036464258 - Tolerance: 0.01